<a href="https://colab.research.google.com/github/aubricot/md-covid-vaccine-finder/blob/main/find_appointments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Find Maryland vaccine appointments at Giant, Safeway, and Mass vaccination sites
*Last modified 13 March 2021*  

Instructions:     
1) Add jeffaukey@gmail.com to your Gmail contacts.  
2) Edit **runtime** and **to_gmail** in form fields to right:   
* **runtime**: number of minutes code should run for (currently set to 180 minutes / 3 hours)
* **to_gmail**: gmail address where you will receive a notification when an appointment becomes available (must be @gmail, will not work as written for other email servers)

Coming soon: Support for CVS appointments

In [ ]:
from bs4 import BeautifulSoup
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from urllib.request import urlopen
import datetime
import pytz
import time
import json

# TO DO: Enter how many minutes to run loop for using form field to right
runtime = 180 #@param
endtime = time.time() + 60 * runtime

# Reload appt portal for runtime mins for specified runtime
while time.time() < endtime:
  try:
      ## 1) Check DMV Vaccine website for Safeway and Mass Vax Sites
      site = "https://www.dmvvaccine.com/"
      # Parse html info from dmvvaccine
      page = urlopen(site)
      html_bytes = page.read()
      html = html_bytes.decode("utf-8")
      #print("html:", html)
      title_index = html.find("<h3>Maryland")
      #print("title index:", title_index)
      start_index = title_index + len("<h3>")
      #print("start index:", start_index)
      end_index = html.find("<h3>Virginia")
      #print("end index:", end_index)
      content = html[start_index:end_index]

      # Inspect content for MD vaccine distrubtion
      #print("Relevant content: \n", content)
      soup = BeautifulSoup(content, 'html.parser')
      #print("Pretty soup style: \n", soup.prettify())
    
      # Get current time
      gmt = datetime.datetime.now()
      est = gmt.astimezone(pytz.timezone('US/Eastern'))

      # Check Safeway appointment availability
      safeway = soup.find_all('div')[6]
      if "Fully Booked" in str(safeway):
        print("Boo, no appointments available at MD Safeway as of ", est)
      else: 
        print("~~YAY! Appointment available at MD Safeway as of ", est)
        sitename = "Safeway"
        url = "https://www.mhealthappointments.com/covidappt"
    
      # Check Mass Vaccination Site appointment availability
      massvaxsts = [soup.find_all('div')[x] for x in [9,12,15]]
      for massvaxst in massvaxsts:
        if 'Fully Booked' in str(massvaxst):
          print("Boo, no appointments available at MD Mass Vaccination Sites as of ", est)
        else:
          print("~~YAY! Appointment available at MD Mass Vaccination Sites as of ", est)
          sitename = "MD Mass Vaccination Sites"
          url = "https://www.adventisthealthcare.com/coronavirus-covid-19/vaccine/"

      ## 2) Try Giant vaccine appointment website
      try:
        site = "https://covidinfo.reportsonline.com/covidinfo/GiantFood.html"
        page = urlopen(site)
        html_bytes = page.read()
        html = html_bytes.decode("utf-8")
        #print("html:", html)
        title_index = html.find("<h2>")
        #print("title index:", title_index)
        start_index = title_index + len("<h2>")
        #print("start index:", start_index)
        end_index = html.find("</h2>")
        #print("end index:", end_index)
        content = html[start_index:end_index]
        #print("title:", content)
        gmt = datetime.datetime.now()
        est = gmt.astimezone(pytz.timezone('US/Eastern'))

        # Check Giant appointment availability
        if "no COVID-19 vaccine appointments" in content:
          print("Boo, no appointments available at MD Giant as of ", est)
        else: 
          print("~~YAY! Appointment available at MD Giant as of ", est)
          sitename = "Giant"
          url = "https://covidinfo.reportsonline.com/covidinfo/GiantFood.html"
      except:
        print("Error accessing Giant vaccine website")

      # If appointment is available, send email notification to to_gmail
      try:
          if sitename:
            server = smtplib.SMTP('smtp.gmail.com', 587)
            server.ehlo()
            server.starttls()
            server.ehlo()
            from_gmail = "jeffaukey@gmail.com" 
            from_pass = "PinkFlower$$1976" 
            # TO DO: enter email address to receive Appt. Alert at in form field to right
            to_gmail = "YourEmailAddress@gmail.com" #@param {type:"string"}
            server.login(from_gmail, from_pass)
            msg = "Subject: Covid-19 vaccine appointment available at {}!\n\nClick here to register: {}\n\nYours Truly, \n\nJeff".format(sitename, url)
            server.sendmail(from_gmail, to_gmail, msg)
            server.quit()
            # Reset sitename and url variables
            sitename = None
            url = None

      except:
          pass
      # Take a 3-sec nap between refreshing vaccine appointment website to limit timeouts
      time.sleep(3)

  # Take a 10-sec nap when get booted
  except: 
      print("Error accessing DMV Vaccine website, taking 10 second nap before re-trying")
      time.sleep(5)

Boo, no appointments available at MD Safeway as of  2021-03-14 17:34:00.665786-04:00
Boo, no appointments available at MD Mass Vaccination Sites as of  2021-03-14 17:34:00.665786-04:00
Boo, no appointments available at MD Mass Vaccination Sites as of  2021-03-14 17:34:00.665786-04:00
Boo, no appointments available at MD Mass Vaccination Sites as of  2021-03-14 17:34:00.665786-04:00
Boo, no appointments available at MD Giant as of  2021-03-14 17:34:00.825574-04:00
Boo, no appointments available at MD Safeway as of  2021-03-14 17:34:03.943307-04:00
Boo, no appointments available at MD Mass Vaccination Sites as of  2021-03-14 17:34:03.943307-04:00
Boo, no appointments available at MD Mass Vaccination Sites as of  2021-03-14 17:34:03.943307-04:00
Boo, no appointments available at MD Mass Vaccination Sites as of  2021-03-14 17:34:03.943307-04:00
Boo, no appointments available at MD Giant as of  2021-03-14 17:34:04.043372-04:00
Boo, no appointments available at MD Safeway as of  2021-03-14 1